## 1. TOKENIZATION - AUTO DETECT

In [ ]:
SELECT 
    flattened.VALUE AS MASKED_VALUES
FROM 
    LATERAL FLATTEN(
        INPUT => protecto.vault.protecto_mask(
            (SELECT ARRAY_AGG(NAME) 
             FROM (SELECT TOP 5 NAME FROM PII_DATA)),
            'None', 
            'None', 
            'token_value'
        )
    ) AS flattened;


## 1.1 TOKENIZATION - AUTO DETECT (TOXICITY ANALYSIS)

In [ ]:
SELECT 
    flattened.VALUE AS TOXICITY_ANALYSIS
FROM 
    LATERAL FLATTEN(
        INPUT => protecto.vault.protecto_mask(
            (SELECT ARRAY_AGG(NAME) 
             FROM (SELECT TOP 5 NAME FROM PII_DATA)),
            'None', 
            'None', 
            'toxicity_analysis'
        )
    ) AS flattened;

## 1.2 TOKENIZATION - AUTO DETECT (TOXICITY)

In [ ]:
SELECT 
    flattened.VALUE AS TOXICITY
FROM 
    LATERAL FLATTEN(
        INPUT => protecto.vault.protecto_mask(
            (SELECT ARRAY_AGG(NAME) 
             FROM (SELECT TOP 5 NAME FROM PII_DATA)),
            'None', 
            'None', 
            'toxicity'
        )
    ) AS flattened;

## 1.3 TOKENIZATION - AUTO DETECT (RAW JSON)

In [ ]:
SELECT protecto.vault.protecto_mask(
    (SELECT ARRAY_AGG(NAME) 
     FROM (SELECT TOP 5 NAME FROM PII_DATA)),
    'None', 
    'None', 
    'raw_json'
);

## 2. TOKENIZATION - FORMAT & TOKEN TYPE

In [ ]:
SELECT 
    flattened.VALUE AS MASKED_VALUES
FROM 
    LATERAL FLATTEN(
        INPUT => protecto.vault.protecto_mask(
            (SELECT ARRAY_AGG(NAME) 
             FROM (SELECT TOP 5 NAME FROM PII_DATA)),
            'Text Token', 
            'Person Name',  
            'token_value'
        )
    ) AS flattened;

## WRTING TO NEW TABLE

In [ ]:
-- Create a new table with the result
CREATE OR REPLACE TABLE PII_MASKED_SAMPLE AS
WITH top_5_names AS (
    SELECT ROW_NUMBER() OVER (ORDER BY NAME) AS idx, NAME
    FROM PII_DATA
    ORDER BY NAME
    LIMIT 5
),
masked_names AS (
    SELECT 
        ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) AS idx, 
        flattened.VALUE AS MASKED_NAME
    FROM 
        LATERAL FLATTEN(
            INPUT => protecto.vault.protecto_mask(
                (SELECT ARRAY_AGG(NAME) FROM top_5_names),
                'Text Token', 
                'Person Name', 
                'token_value'
            )
        ) AS flattened
)

SELECT 
    t.NAME,
    m.MASKED_NAME
FROM 
    top_5_names t
JOIN 
    masked_names m
ON 
    t.idx = m.idx;


In [ ]:
SELECT * FROM PII_MASKED_SAMPLE;

## 3. PROTECTO UNMASK

In [ ]:
SELECT 
    flattened.VALUE AS ORIGNAL_VALUES
FROM 
    LATERAL FLATTEN(
        INPUT => protecto.vault.protecto_unmask(
            SELECT ARRAY_AGG(MASKED_NAME) FROM PII_MASKED_SAMPLE
        )
    ) AS flattened;

## 3.1 PROTECTO UNMASK - COMPARE

In [ ]:
WITH top_5_names AS (
    SELECT ROW_NUMBER() OVER (ORDER BY MASKED_NAME) AS idx, MASKED_NAME
    FROM PII_MASKED_SAMPLE
    ORDER BY MASKED_NAME
    LIMIT 5
),
masked_names AS (
    SELECT 
        ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) AS idx, 
        flattened.VALUE AS ORIGINAL_NAME
    FROM 
        LATERAL FLATTEN(
            INPUT => protecto.vault.protecto_unmask(
                SELECT ARRAY_AGG(MASKED_NAME) FROM top_5_names
            )
        ) AS flattened
)

SELECT 
    t.MASKED_NAME,
    m.ORIGINAL_NAME
FROM 
    top_5_names t
JOIN 
    masked_names m
ON 
    t.idx = m.idx;